# 2020/09/26 定期勉強会 データ分析演習

## ライブラリの読み込み
<br>
分析に使用するライブラリを読み込みます。ライブラリを読み込んでおくことで、できることが増えます。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as sp
from sklearn.datasets import load_boston

## データの読み込み
<br>
今回はライブラリのデータを使用します。
<br>
ライブラリを読み込んでいたおかげで、データを簡単に読み込めるようになりました。

In [ ]:
data = load_boston()

In [ ]:
# データを扱いやすいようにpandasのdata.frameに変換する
df = pd.DataFrame(data.data, columns = data.feature_names).assign(MEDV=np.array(data.target))

### データの軽い確認
<br>
データの頭5行何列か、という構造を軽く確認します。

In [ ]:
df.head()

### 変数の説明

| カラム | 説明 |
| ---- | ---- |
| CRIM | 町ごとの一人当たりの犯罪率 |
| ZN | 宅地の比率が25,000平方フィートを超える敷地に区画されている。 |
| INDUS |	町当たりの非小売業エーカーの割合 |
| CHAS | チャーリーズ川ダミー変数（川の境界にある場合は1、それ以外の場合は0） |
| NOX | 一酸化窒素濃度（1000万分の1） |
| RM | 1住戸あたりの平均部屋数 |
| AGE | 1940年以前に建設された所有占有ユニットの年齢比率 |
| DIS | 5つのボストンの雇用センターまでの加重距離 |
| RAD | ラジアルハイウェイへのアクセス可能性の指標 |
| TAX |	10,000ドルあたりの税全額固定資産税率 |
| PTRATIO |	生徒教師の比率 |
| B | 町における黒人の割合 |
| LSTAT | 人口当たり地位が低い率 |
| MEDV | 1000ドルでの所有者居住住宅の中央値 |

## 仮説の確認
<br>
仮説：住宅価格(1000$)は、「非小売業エーカーの割合(%)」、「平均部屋数(部屋)」、「税全額固定資産税率(部屋)」、「生徒教師の比率(%)」、「地位が低い人の割合(%)」によって算出される。

## 使用する変数
<br>

| 目的変数（予測したい値） | 説明 |
| ---- | ---- |
| MEDV | 住宅価格 |

<br>

| 説明変数（予測したい値に影響を与える変数） | 説明 |
| ---- | ---- |
| RM | 1住戸あたりの平均部屋数 |
| PTRATIO |	生徒教師の比率 |

## データ収集
<br>
仮説に沿ったデータのみを抽出する。

In [ ]:
df = df[['RM', 'PTRATIO', 'MEDV']]

### データの確認
<br>
変数の数とデータ件数の確認。

In [ ]:
print('(行数, 列数) = ', df.shape)

データの情報を確認する。

In [ ]:
df.info()

### 欠損値の確認

In [ ]:
print('Train columns with null values: {} \n' .format(df.isnull().sum()))

### データの基礎統計量の確認
<br>

| 項目名 | 説明 |
| ---- | ---- |
| count |	要素の個数 |
| mean |	平均 |
| std |	標準偏差 |
| min |	最小値 |
| 25% |	第一四分位数 |
| 50% |	中央値 |
| 75% |	第三四分位数 |
| max |	最大値 |

In [ ]:
df.describe()

相関係数の算出

In [ ]:
df_corr = df_corr = df.corr()
display(df_corr)

相関係数を見やすく。

In [ ]:
sns.set_context('talk')
fig = plt.subplots(figsize = (8, 8))
 
sns.heatmap(df_corr, annot = True, fmt = '.2f', cmap = 'Blues', square = False)

全体のヒストグラムと散布図の描画。

In [ ]:
sns.pairplot(df)

### 目的変数のみ確認

ヒストグラム描画

In [ ]:
%matplotlib inline
df['MEDV'].hist(bins = 50, figsize = (10, 7.5))
plt.rcParams['figure.figsize'] = [20, 15]

箱ひげ図描画

### 説明変数のみ確認

ヒストグラム描画

In [ ]:
%matplotlib inline
df[['RM', 'PTRATIO']].hist(bins=50, figsize=(15,7.5))
plt.rcParams['figure.figsize'] = [20, 15]

## データ加工

### 外れ値対処

In [ ]:
%matplotlib inline
fig_o, ax_o = plt.subplots(figsize=(9.6, 5))
ax_o.set_title('Box Plot')
ax_o.boxplot(df['PTRATIO'], vert = False,  labels=['PTRATIO'])
plt.rcParams['figure.figsize'] = [20, 15]

外れ値の除去。

In [ ]:
df.query('PTRATIO >= 13', inplace = True)

In [ ]:
%matplotlib inline
df['PTRATIO'].hist(bins = 50, figsize = (10, 7.5))
plt.rcParams['figure.figsize'] = [20, 15]

In [ ]:
df_corr = df_corr = df.corr()
display(df_corr)

In [ ]:
sns.set_context('talk')
fig = plt.subplots(figsize = (8, 8))
 
sns.heatmap(df_corr, annot = True, fmt = '.2f', cmap = 'Blues', square = True)

In [ ]:
sns.pairplot(df)

## 回帰分析

一旦データを再確認する。

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

### モデル構築
<br>

| 目的変数 | 説明 |
| ---- | ---- |
| MEDV | 住宅価格 |

<br>

| 説明変数 | 説明 |
| ---- | ---- |
| RM | 1住戸あたりの平均部屋数 |
| PTRATIO |	生徒教師の比率 |

<br>

#### モデル式

$ {\rm MEDV} = \beta_0 + {\rm RM} \: \times \beta_1 + {\rm PTRATIO} \: \times \beta_2 $

In [ ]:
x = df[['RM', 'PTRATIO']]
y = df['MEDV']
X = sm.add_constant(x)

model = sm.OLS(y, X)
result = model.fit()
result.summary()

### 着目する値
<br>
・Adj. R-squared: 自由度調整済み決定係数

・Prob (F-statistic): モデルに対するF検定の有意確率

・coef: 偏回帰係数

・P>|t|: 偏回帰係数に対するt検定の有意確率

・（必要に応じて）AIC: 赤池情報量基準

・（必要に応じて）BIC: ベイズ情報量基準

## 影響度比較

In [ ]:
x = df[['RM', 'PTRATIO']]
y = df['MEDV']
y.reset_index(inplace=True, drop=True)
x_n = pd.DataFrame(sp.stats.zscore(x, axis=0), \
                   columns=['RM', 'PTRATIO'])
X_n = sm.add_constant(x_n)

model = sm.OLS(y, X_n)
result = model.fit()
result.summary()